# Marc tools

## Import

In [ ]:
import os
import plotly
import importlib
import re
import h5py

import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go
import marc_tools as mt
import plotly.express as px

from pathlib import Path
from IPython.display import clear_output

# Set up plotly
plotly.offline.init_notebook_mode()

pio.templates['custom'] = go.layout.Template(
    layout_paper_bgcolor='rgba(0,0,0,0)',
    layout_plot_bgcolor='rgba(0,0,0,0)',
    ) 

# Change background color if using VSC
if 'VSCODE_CWD' in os.environ:
    pio.templates.default = 'plotly+custom'

# Pandas decimal precision
pd.set_option("display.precision", 2)

In [ ]:
# Define folders
project_dir = Path(r'C:\Projects\1993_Pierce_GI_riser')
bflex_dir = project_dir / 'Bflex/Local/Top_300bar'

### BFLEX - Stress and curvature

In [ ]:
# Find curvature
for curv in bflex_dir.glob(r'*elcurvY.mpf'):
    df_curv = mt.bflex.mpf_reader(curv)

df_curv.columns = ['curv']

# Find mpf_files
df_mpf = pd.DataFrame()

for mpf in bflex_dir.glob(r'*nosl?.mpf'):
    
    df_temp = mt.bflex.mpf_reader(mpf)
    df_mpf = pd.concat([df_mpf, df_temp], axis=1)

# Manipulate df
df = pd.DataFrame()

for i in range(17, 33):
    df[f'P{i}'] = df_mpf.filter(regex=f'{i}').max(axis=1)

df_stress = pd.concat([df_curv, df], axis=1)

# Interpolate curvature
df = pd.DataFrame({'curv' : [i/100 for i in range(0, 21)]})

for col in df_stress.columns[1:]:
    df[col] = np.interp(df.curv, df_stress.curv, df_stress[col])

# df_new.columns = ['Curvature'] + [i*22.5 for i in range(0, 16)]
pd.set_option("display.precision", 2)
df_bflex = df

In [ ]:
# Plotting
fig_bf = go.Figure()

for inc in range(0, 21, 5):
    fig_bf.add_trace(go.Scatterpolar(r=df.iloc[inc, 1:],
                                  theta=[-90 + 22.5*i for i in range(0, 17, 1)],
                                  fill='toself',
                                  name=f'Curv {df.curv.loc[inc]}'
                                  ))

fig_bf.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 600]
        )),
    showlegend=True
)

fig_bf.update_layout(title='Stress BFLEX',
                  autosize=False,
                  font=dict(color='white'),
                  width=1000, height=800)

fig_bf.show()

### Marc

#### Stress

In [ ]:
# Bad incs
bad_incs = [26, 28, 29, 38, 39, 40, 46, 47, 51, 57, 64, 65, 67]

# Read Marc files
marc_dir = Path(r'\\nas-ask-001\Projects\1993_Pierce_GIR_fatigue\Marc\jupyter_notebook')
marc_files = marc_dir.glob('*stresses_inner_ta_inc*[0-9].csv*')

# Read list with coordinates per node
df_list = pd.read_csv(marc_dir / 'angles.csv').iloc[:,1:]
df_list.drop('node_index', axis=1, inplace=True)

# Iterate over files and create dataframe
df = pd.DataFrame()

for mf in marc_files:
    # Read increment
    inc = int(re.findall(r'\d+', mf.stem)[0])

    if inc in bad_incs:
        continue
    
    # Read dataframe and add x, y and z coordinate
    df_read = pd.read_csv(mf, index_col=None).iloc[:,1:]

    if all(df_read['node'] == df_list['node_id']):
        df_read = pd.concat([df_read, df_list[['z', 'theta']]], axis=1)

    df = pd.concat([df, df_read], axis=0)

df.sort_values(['inc', 'z', 'theta'], inplace=True)


In [ ]:
df_marc = pd.DataFrame()

for inc in range(0, 70, 10):
    
    if inc < 10:
        inc_start = inc
        inc_end = inc + 10
    else:
        inc_start = inc + 1
        inc_end = inc + 10

    # print(f'Checking increments {inc_start} <= inc <= {inc_end}')

    dfn = df[(df.inc <= inc_end) & (df.inc >= inc_start)]

    dfn['diff'] = dfn.groupby(['inc', 'z'])['theta'].diff().gt(3).cumsum()

    dfg = dfn.groupby(['inc', 'z', 'diff']).max()
    dfg.reset_index(inplace=True)
    dfg.drop(['diff'], axis=1, inplace=True)

    df_marc = pd.concat([df_marc, dfg], axis=0)

# Turn around 90 degrees to orient with bflex
df_marc['theta'] = df_marc['theta'] - 90

# Round z
df_marc['z'] = [int(i) for i in np.round(list(df_marc['z']))]
df_marc['inc'] = [int(i) for i in df_marc['inc']]
df_marc['node'] = [int(i) for i in df_marc['node']]

# Space saving
df_marc = df_marc.astype({'theta': 'float32', 'stress': 'float32'})

In [ ]:
# Plotting
fig_marc = go.Figure()

for inc in range(0, 71, 10):
    df_plot = df_marc[(df_marc.inc==inc) & (df_marc.z==2209)]
    fig_marc.add_trace(go.Scatterpolar(r=df_plot['stress'][::3],
                                  theta=df_plot['theta'][::3],
                                  fill='toself',
                                  name=f'Inc {inc}'
                                  ))

fig_marc.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 500]
        )),
    showlegend=True
)

fig_marc.update_layout(title='Stress Marc',
                  autosize=False,
                  font=dict(color='white'),
                  width=1000, height=800)

fig_marc.show()

## CrossSection - Read files

In [596]:
project_folder = Path(r'V:\1993_Pierce_GIR_fatigue\Marc\analysis')
runs = ['CS'] # Folder with runs
exclude_cases = [] # optional - cases to be excluded from post-processing

df_case = mt.crossection.df_cs(project_folder, runs, positions='start')

In [597]:
# Marc list
marc_list = sorted(list(set(df_marc['z'])))

# Cross section list
cs_list = [int(k) for k in df_case['nov_8inch_gir_3p_with_bs_g3_job1'].keys()]

# Find closest values
cs_dict = {}
for cs in cs_list:
    cs_dict[cs] = min(marc_list, key=lambda x: abs(x - cs))

# Add column
df_marc['curv'] = np.nan

# Reduce dataframe to points where CS data is available
for i, (cs_pos, marc_pos) in enumerate(cs_dict.items(), 1):
    print(f'{i/len(cs_dict)*100:.1f} %')
    for inc in range(0, 71, 1):
        df_marc.loc[(df_marc.z==marc_pos) & (df_marc.inc==inc), 'curv'] = df_case['nov_8inch_gir_3p_with_bs_g3_job1'][str(cs_pos)]['K3_4'][inc]
    clear_output(wait=True)

# Remove rows where curv is na
df_marc = df_marc[df_marc.curv.notna()]

30.3


In [594]:
print(f'{20/89*100:.1f} %')

22.5 %


### Create SCF

In [ ]:
df_scf = pd.DataFrame()
bflex_name = {-90 + 22.5*i: f'P{17+i}' for i in range(0, 16, 1)}

for i, marc_pos in enumerate(sorted(list(set(df_marc['z']))), 1):
    print(f'Pos {i} of {len(list(set(df_marc["z"])))}')

    # Find dictionary with bflex vs marc angles
    theta_dict = {}
    dft = df_marc[(df_marc.z==marc_pos) & (df_marc.inc==0)]

    for bf_angle in [-90 + 22.5*i for i in range(0, 16, 1)]:
        theta_dict[bf_angle] = min(dft.theta, key=lambda x: abs(x - bf_angle))

    for inc in range(0, 71, 1):
        # Dont go through bad increments (no input?)
        if inc in bad_incs:
            continue

        # Create dataframe containing specific position and inc only
        dft = df_marc[(df_marc.z==marc_pos) & (df_marc.inc==inc)]

        # Find stress for correct curvature and angle
        df_temp = dft[dft.theta.isin(theta_dict.values())]

        # Find corresponding bflex stress
        curv = df_temp.curv.iloc[0]
        df_interp = df_bflex.iloc[(df_bflex.curv - curv).abs().argsort()[:2]]
        y1, y2 = df_interp.iloc[0,1:], df_interp.iloc[1,1:]

        linfit = scipy.interpolate.interp1d(df_interp.curv, np.vstack([y1, y2]), axis=0)
        df_temp['stress_bflex'] = linfit(curv)

        # Calculate the SCF
        df_temp['scf'] = df_temp.stress / df_temp.stress_bflex

        df_scf = pd.concat([df_scf,
                            df_temp[['inc', 'z', 'theta', 'curv', 'stress',
                                     'stress_bflex', 'scf']]
                            .reset_index(drop=True)], axis=0)
        clear_output(wait=True)


In [ ]:
df_scf[(df_scf.inc==0) & (df_scf.z==positions[pos])]

## Fix so that positions refer to first position

In [ ]:
# Plotting
fig_scf = go.Figure()
positions = sorted(list(set(df_scf.z)))
pos = 25

for inc in range(0, 71, 10):

    # Extrat specific positinos
    df_plot = df_scf[(df_scf.inc==inc) & (df_scf.z==positions[pos])]
    print(df_plot.curv.iloc[0])

    # Find curvature
    curv = df_plot.curv.iloc[0]

    # Plot
    fig_scf.add_trace(go.Scatterpolar(r=df_plot['scf'][::3],
                                  theta=df_plot['theta'][::3],
                                  fill='toself',
                                  name=f'Curvature {curv}'
                                  ))

fig_scf.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1.5]
        )),
    showlegend=True
)


fig_scf.update_layout(title=f'Stress correction factor @ {positions[pos]} mm',
                      autosize=False,
                      font=dict(color='white'),
                      width=1000, height=800)

fig_scf.show()

## Examples

In [ ]:
# Plotly 3d surface with color - Example
df_plot = df_marc.where(df_marc['inc']==60).dropna()

df_plot['x'] = 200.0 * np.sin(df_plot['theta'])
df_plot['y'] = 200.0 * np.cos(df_plot['theta'])

fig = go.Figure()

fig.add_trace(go.Surface(
    x=df_plot['x'].tolist()[0:100],
    y=df_plot['y'].tolist()[0:100],
    z=[df_plot['z'].tolist()[0:100]]*100,
    surfacecolor=df_plot['stress'].tolist()[0:10]
))
fig.update_layout(title='Stress', autosize=True,
                  width=1000, height=800)

fig.update_layout(scene=dict(xaxis=dict(range=[-200, 200]),
                             yaxis=dict(range=[-200, 200]),
                             zaxis=dict(range=[0, 4000])
))
fig.show()

In [ ]:
# Plotly express
fig = px.line_polar(df_marc, r='stress', theta='theta', line_close=True)
fig.update_traces(fill='toself')
fig.show()

## Read marc input

In [ ]:
mf = Path(r'\\nas-ask-001\Projects\1993_Pierce_GIR_fatigue\Marc\analysis\NOV_8inch_GIR_3p_with_bs_g3_job1.h5')

with h5py.File(mf, 'r') as f:
    print(f'Keys {f["Marc"]["Results"]["Element"]["Tensor"]["Cauchy Stress in Preferred Sys"].keys()}')
